<a href="https://colab.research.google.com/github/Lehula1/divergGit/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install plotly
! pip install numpy
! pip install yfinance
! pip install pandas_ta
! pip install matplotlib
! pip install ta
! pip install finta
! pip install pytickersymbols

! pip install beautifulsoup4
! pip install requests
! pip install datetime

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly
from pandas_datareader import data as pdr
import pandas_ta as ta
import yfinance as yf
from scipy.signal import argrelextrema
import plotly.graph_objects as go
from collections import deque
from ta.momentum import StochasticOscillator
from plotly.subplots import make_subplots
import datetime as dt

import bs4 as bs
import requests
import datetime

import requests
from bs4 import BeautifulSoup
import yfinance as yf







def get_stock_list():
  # this is the website we're going to scrape from
  url = "https://www.malaysiastock.biz/Stock-Screener.aspx"
  response = requests.get(url, headers={'User-Agent':'test'})
  soup = BeautifulSoup(response.content, "html.parser")
  table = soup.find(id = "MainContent2_tbAllStock")
  # return the result (only ticker code) in a list
  return [stock_code.get('href')[-4:] for stock_code in table.find_all('a')]

def get_stock_price(code):
  # you can change the start date
  data = yf.download(code, start="2021-01-01")
  return data

def add_EMA(price, day):
  return price.ewm(span=day).mean()

def add_STOCH(close, low, high, period, k, d=0):
    STOCH_K = ((close - low.rolling(window=period).min()) / (high.rolling(window=period).max() - low.rolling(window=period).min())) * 100
    STOCH_K = STOCH_K.rolling(window=k).mean()
    if d == 0:
      return STOCH_K
    else:
      STOCH_D = STOCH_K.rolling(window=d).mean()
      return STOCH_D

def check_bounce_EMA(df):
  candle1 = df.iloc[-1]
  candle2 = df.iloc[-2]
  cond = candle1['STOCH_%K(14,3,3)'] <= 10

  return cond


# a list to store the screened results
screened_list = []
# get the full stock list
stock_list = get_stock_list()
for stock_code in stock_list:

  print(stock_code) # remove this if you dont want the ticker to be printed
  try:
    # Step 1: get stock price for each stock
    price_chart_df = get_stock_price(stock_code + ".KL")

    # Step 2: add technical indicators (in this case EMA)
    close = price_chart_df['Close']
    low = price_chart_df['Low']
    open = price_chart_df['Open']
    high = price_chart_df['High']
    price_chart_df['STOCH_%K(14,3,3)'] = add_STOCH(close, low, high, 14, 3)


    # if all 3 conditions are met, add stock into screened list
    if check_bounce_EMA(price_chart_df):
      screened_list.append(stock_code)
      print(screened_list)
  except Exception as e:
    print(e)

In [21]:
import bs4 as bs
import requests
import yfinance as yf
import datetime

resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]

start = datetime.datetime(2021, 1, 1)
end = datetime.datetime(2022, 1, 1)
data = yf.download(tickers, period="1day")

df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.set_index('Date', inplace=True)
df.to_csv('sp500_stocks.csv')

print(df)



"""
data1 = yf.download(tickers='SPY', period='30d', interval='1d')

fig = go.Figure(data=[go.Candlestick(x=data1.index,
                                     open=data1['Open'],
                                     high=data1['High'],
                                     low=data1['Low'],
                                     close=data1['Close'])])


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()
"""


[*********************100%%**********************]  503 of 503 completed

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: Exception("%ticker%: Period '1day' is invalid, must be one of ['1mo', '3mo', '6mo', 'ytd', '1y', '2y', '5y', '10y', 'max']")
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No data found, symbol may be delisted')



           Symbol   Adj Close       Close        High         Low        Open  \
Date                                                                            
2024-01-08      A  133.380005  133.380005  133.570007  129.809998  130.139999   
2024-01-08    AAL   14.580000   14.580000   14.670000   14.020000   14.030000   
2024-01-08   AAPL  185.559998  185.559998  185.600006  181.500000  182.089996   
2024-01-08   ABBV  161.429993  161.429993  162.770004  159.779999  162.179993   
2024-01-08   ABNB  140.080002  140.080002  140.250000  136.610001  137.309998   
...           ...         ...         ...         ...         ...         ...   
2024-01-08    YUM  128.639999  128.639999  128.779999  127.180000  128.210007   
2024-01-08    ZBH  121.970001  121.970001  122.779999  120.519997  120.629997   
2024-01-08   ZBRA  261.089996  261.089996  262.279999  251.360001  252.110001   
2024-01-08   ZION   44.540001   44.540001   44.590000   43.400002   43.779999   
2024-01-08    ZTS  196.1499

"\ndata1 = yf.download(tickers='SPY', period='30d', interval='1d')\n\nfig = go.Figure(data=[go.Candlestick(x=data1.index,\n                                     open=data1['Open'],\n                                     high=data1['High'],\n                                     low=data1['Low'],\n                                     close=data1['Close'])])\n\n\nfig.update_layout(xaxis_rangeslider_visible=False)\nfig.show()\n"

In [ ]:
import bs4 as bs
import requests
import yfinance as yf
import datetime

#I am using a csv file with a list of all the tickers which I use to create a pandas dataframe and form a space seperated string of all of the tickers called all_symbols
#I have simplified the pandas dataframe to a list for the purpose of this question


resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

ticker_list = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    ticker_list.append(ticker)

ticker_list = [s.replace('\n', '') for s in ticker_list]

#--------------------------------------------

#ticker_list = ['AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CDAY', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DHI', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EG', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FI', 'FLT', 'FMC', 'F', 'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GEHC', 'GEN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JBL', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PANW', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC', 'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UBER', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VTRS', 'VICI', 'V', 'VMC', 'WRB', 'WAB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']
all_symbols  = " ".join(ticker_list)

tickers = yf.Tickers(all_symbols)

for ticker in ticker_list:
    if (ticker != 'BRK.B') and (ticker != 'BF.B'):
      price = tickers.tickers[ticker].info["currentPrice"]
      print(ticker, price)

In [44]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
countries = stock_data.get_all_countries()
indices = stock_data.get_all_indices()
industries = stock_data.get_all_industries()

#You can select all stocks of an index as follows:

from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
german_stocks = stock_data.get_stocks_by_index('DAX')
uk_stocks = stock_data.get_stocks_by_index('FTSE 100')
us_stocks = stock_data.get_stocks_by_index('S&P 500')

#print(list(us_stocks))

#If you are only interested in ticker symbols, then you should have a look at the following lines:

from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
# the naming conversation is get_{index_name}_{exchange_city}_{yahoo or google}_tickers
dax_google = stock_data.get_dax_frankfurt_google_tickers()
dax_yahoo = stock_data.get_dax_frankfurt_yahoo_tickers()
sp100_yahoo = stock_data.get_sp_100_nyc_yahoo_tickers()
sp500_google = stock_data.get_sp_500_nyc_google_tickers()
dow_yahoo = stock_data.get_dow_jones_nyc_yahoo_tickers()
# there are too many combination. Here is a complete list of all getters
all_ticker_getter_names = list(filter(
   lambda x: (
         x.endswith('_google_tickers') or x.endswith('_yahoo_tickers')
   ),
   dir(stock_data),
))
print(all_ticker_getter_names)

['get_aex_frankfurt_google_tickers', 'get_aex_frankfurt_yahoo_tickers', 'get_aex_london_google_tickers', 'get_aex_london_yahoo_tickers', 'get_aex_moscow_google_tickers', 'get_aex_moscow_yahoo_tickers', 'get_aex_nyc_google_tickers', 'get_aex_nyc_yahoo_tickers', 'get_bel_20_frankfurt_google_tickers', 'get_bel_20_frankfurt_yahoo_tickers', 'get_bel_20_london_google_tickers', 'get_bel_20_london_yahoo_tickers', 'get_bel_20_moscow_google_tickers', 'get_bel_20_moscow_yahoo_tickers', 'get_bel_20_nyc_google_tickers', 'get_bel_20_nyc_yahoo_tickers', 'get_cac_40_frankfurt_google_tickers', 'get_cac_40_frankfurt_yahoo_tickers', 'get_cac_40_london_google_tickers', 'get_cac_40_london_yahoo_tickers', 'get_cac_40_moscow_google_tickers', 'get_cac_40_moscow_yahoo_tickers', 'get_cac_40_nyc_google_tickers', 'get_cac_40_nyc_yahoo_tickers', 'get_cac_mid_60_frankfurt_google_tickers', 'get_cac_mid_60_frankfurt_yahoo_tickers', 'get_cac_mid_60_london_google_tickers', 'get_cac_mid_60_london_yahoo_tickers', 'get_ca

In [2]:
ticker = 'ACLS'
yfObj = yf.Ticker(ticker)
data = yfObj.history(period='3mo', interval='1d')

close = data['High'].values
dates = data.index

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

max_idx = argrelextrema(data['High'].values, np.greater, order=2)[0]
min_idx = argrelextrema(data['Low'].values, np.less, order=2)[0]

PriceH = 0
PriceHH = 0
# Get K consecutive higher peaks
K = 2
highs = data.iloc[max_idx]['High']

extrema = []
count = 0
ex_deque = deque(maxlen=K)
for i, idx in enumerate(max_idx):
  if i == 0:
    ex_deque.append(idx)
    continue
  if highs[i] < highs[i-1]:
    ex_deque.clear()

  ex_deque.append(idx)

  if len(ex_deque) == K:
    # K-consecutive higher highs found
    extrema.append(ex_deque.copy())
    #extrema count++++++++++++++++++++++++++++++++++++++++++++++++
    #print(extrema[count][0])
    #print(extrema[count][1])
    count += 1

test = data.iloc[extrema[0][0]]['High']
"""
#print(highs)
print(data.iloc[max_idx])
print(ex_deque)
print(test)
print(extrema.index)
print(extrema[0][1])
print(extrema[1])
print(extrema[-1][1])
print(extrema)
print(max_idx)
print(data.index[max_idx])
print(data.iloc[max_idx,1])
"""

data.ta.stoch(high='high', low='low', k=14, d=3, append=True)

# Avoid case-sensitive issues for accessing data.
# Optional if using pandas_ta


data.columns = [x.lower() for x in data.columns]

max_stoch = argrelextrema(data['stochk_14_3_3'].values, np.greater, order=2)[0]


stoch_highs = data.iloc[max_stoch]['stochk_14_3_3']


extremaStoch = []
count1 = 0
stoch_deque = deque(maxlen=K)
for j, sto in enumerate(max_stoch):
  if j == 0:
    stoch_deque.append(sto)
    continue
  if stoch_highs[j] > stoch_highs[j-1] or data['high'].values[max_stoch[j-1]] > data['high'].values[max_stoch[j]]:

      stoch_deque.clear()
      print(data['high'].values[max_stoch[j-1]])
      print(data['high'].values[max_stoch[j]])


  stoch_deque.append(sto)

  if len(stoch_deque) == K:
    # K-consecutive higher highs found
    extremaStoch.append(stoch_deque.copy())


    #print(extremaStoch[count1][0])

    #print(extremaStoch[count1][1])
    count1 += 1







"""
candlestickHigh = data['high'].values[max_stoch[1]]
test = argrelextrema(data['high'].values, np.greater, order=2)[0]
print(candlestickHigh)
print(close)
print(extremaStoch)
print(stoch_highs)
print(data.iloc[extremaStoch[1]])

print(data['high'].values)
print(highs)
print(max_idx)

print(data['stochk_14_3_3'].values) #----------------------------------------------------------------------
print(data.iloc[max_stoch]['stochk_14_3_3'])

"""

# Create our primary chart
# the rows/cols arguments tell plotly we want two figures
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,row_heights=[1,.5], vertical_spacing = 0.01)

# Create our Candlestick chart with an overlaid price line
fig.append_trace(
    go.Candlestick(
        x=data.index,
        open=data['open'],
        high=data['high'],
        low=data['low'],
        close=data['close'],
        showlegend=False
    ), row=1, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_idx],
        y=data.iloc[max_idx,1]+5.5,
        marker=dict(
            symbol='cross',
            color='green',
            size=10)))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[min_idx],
        y=data.iloc[min_idx,2]-5.5,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))

# Fast Signal (%k)
fig.append_trace(
    go.Scatter(
        x=data.index,
        y=data['stochk_14_3_3'],
        line=dict(color='#ff9900', width=2),
        name='fast'), row=2, col=1)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_stoch],
        y=data.iloc[max_stoch]['stochk_14_3_3'],
        marker=dict(
            symbol='cross',
            color='red',
            size=10)), row=2, col=1)

fig.add_trace(
    go.Scatter(
        x=[data.index[extremaStoch[-1][0]], data.index[extremaStoch[-1][1]]],
        y=[data.iloc[extremaStoch[-1][0],7], data.iloc[extremaStoch[-1][1],7]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')),row=2, col=1)




# Extend our y-axis a bit
fig.update_yaxes(range=[-10, 110], row=2, col=1)

# Add upper/lower bounds
fig.add_hline(y=0, col=1, row=2, line_color="#666", line_width=2)
fig.add_hline(y=100, col=1, row=2, line_color="#666", line_width=2)

# Add overbought/oversold
fig.add_hline(y=20, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')
fig.add_hline(y=80, col=1, row=2, line_color='#336699', line_width=2, line_dash='dash')

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"])] #hide weekends
)

fig.update_layout(xaxis_rangeslider_visible=False,  width=1500,
     margin=dict(
        l=50,
        r=50,
        b=10,
        t=10,
        pad=4))



# View our chart in the system default HTML viewer (Chrome, Firefox, etc.)


NameError: ignored

In [ ]:
# Add some indicators


fig=go.Figure()

fig = plotly.subplots.make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01,
                    row_heights=[0.5,0.4])

fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close']))

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[max_idx],
        y=data.iloc[max_idx,1]+2,
        marker=dict(
            symbol='cross',
            color='green',
            size=10,

        )
    )
)

fig.add_trace(
    go.Scatter(
        mode='markers',
        x=data.index[min_idx],
        y=data.iloc[min_idx,2]-2,
        marker=dict(
            symbol='cross',
            color='red',
            size=10)))

fig.add_trace(
    go.Scatter(
        x=[data.index[extrema[-1][0]], data.index[extrema[-1][1]]],
        y=[data.iloc[extrema[-1][0],1], data.iloc[extrema[-1][1],1]],
        mode='lines',
        name='trend line',
        line=dict(color='blue')))




fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"])] #hide weekends
)

fig.update_layout(xaxis_rangeslider_visible=False)



In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(data['High'])
_ = [plt.plot(dates[i], close[i], c=colors[1]) for i in extrema]


#create figure
plt.figure(figsize=(20,5))

plt.scatter(data.iloc[max_idx].index, data.iloc[max_idx]['High']+.5,
  label='Maxima', s=100, color='green', marker='+')
plt.scatter(data.iloc[min_idx].index, data.iloc[min_idx]['Low']-.5,
  label='Minima', s=100, color='red', marker='+')

#define width of candlestick elements
width = .6
width2 = .1

#define up and down prices
up = data[data['Close']>=data['Open']]
down = data[data['Close']<data['Open']]

#define colors to use
col1 = 'green'
col2 = 'red'

#plot up prices
plt.bar(up.index,up['Close']-up['Open'],width,bottom=up['Open'],color=col1)
plt.bar(up.index,up['High']-up['Close'],width2,bottom=up['Close'],color=col1)
plt.bar(up.index,up['Low']-up['Open'],width2,bottom=up['Open'],color=col1)

#plot down prices
plt.bar(down.index,down['Close']-down['Open'],width,bottom=down['Open'],color=col2)
plt.bar(down.index,down['High']-down['Open'],width2,bottom=down['Open'],color=col2)
plt.bar(down.index,down['Low']-down['Close'],width2,bottom=down['Close'],color=col2)

#rotate x-axis tick labels
plt.xticks(rotation=45, ha='right')

#lines
_ = [plt.plot(dates[i], close[i], c=colors[1]) for i in extrema]

plt.legend()
#display candlestick chart
plt.show()

KeyError: ignored

<Figure size 1500x800 with 0 Axes>